## Аггрегация разметки датасета SimpleAr
*Subset edition*

Аггрегация строится по следующей системе:

1. Сбор размеченных пулов с Толоки. Возможны варианты:
    - только общий пул нужно аггрегировать, тогда забирается только он
    - часть данных находится в контрольных заданиях и экзамене, тогда к основному пулу добавляются данные задания
2. Фильтрация разметчиков:
    - в общем пуле есть некоторое количество заранее размеченных заданий - контрольных
    - хорошим считается разметчик, который показывает `accuracy >= 0.5` на данных заданиях
    - формируется список "плохих" разметчиков
3. Аггрегация ответов разметчиков по заданиям:
    - форматирование в заданиях может отличаться от изначального из-за выгрузки с Толоки
    - учитываются только ответы "хороших" разметчиков
    - аггрегация по подготовленным пулам - создается массив карточек вида {key: value}, где key - кортеж из всех значимых элементов задания, value - список из кортежей вида (user_id, answer)
4. Голосование большинством по каждому заданию:
    - минимально необходимое большинство составляет 3 голоса, так как такое большинство валидно для перекрытия 5
    - по результату формируется датафрейм с заданиями и ответами
5. Подгрузка оригинальных данных с разметкой в виде таблицы с заданиями и ответами
6. Соединение таблиц:
    - очистка форматирования в таблице с ответами разметчиков и в таблице с правильными ответами
    - создание единых столбцов с полным заданием
    - соединение таблиц по данному столбцу
    - валидация размеров
7. Подсчет метрик

In [1]:
import pandas as pd
from collections import Counter

### Сбор данных разметки и фильтрация разметчиков

Данный датасет представляет из себя лишь подмножество, которое было специально сгенерировано. Это не тестовый сет датасета из мультимодального бенчмарка (инструкционный сет)! Размер выборки - 200 объектов.

In [2]:
assignments = pd.read_csv('assignments_from_pool_41864367__20-10-2023.tsv', sep='\t')

Разметчикам предлагалось решить математический пример.

Вход: 
- INPUT:formula (пример: `(2 + (-1))`).

Выход:
- OUTPUT:result (целое число, например: `1`).

In [3]:
assignments.head(1)

,INPUT:formula,OUTPUT:result,GOLDEN:result,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:task_suite_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started,ASSIGNMENT:submitted,ASSIGNMENT:accepted,ASSIGNMENT:reward
0,8920 + 2043,10963,NaN,NaN,NaN,https://platform.toloka.ai/task/41864367/00027...,00027eccaf--6532b18902a2ad7a848d2d04,00027eccaf--6532b5a51a56d0600c64f94f,00027eccaf--6532b5a51a56d0600c64f94b,4ff3bb526bef2dfc09d25d66edcdc772,APPROVED,2023-10-20T17:15:17.508,2023-10-20T17:15:47.324,2023-10-20T17:15:47.324,0.029


Фильтруем толокеров с `accuracy < 0.5` на контрольных заданиях, чтобы не учитывать их ответы при подсчете метрик.

In [4]:
from collections import defaultdict

users_dict = defaultdict(lambda: defaultdict(int))

for idx, row in assignments.iterrows():
    formula = row[0]

    out = row[1]
    
    gold = row[2]

    user = row[9]

    if str(user) != "nan" and str(gold) != "nan":
        if int(out) == int(gold):
            users_dict[user]["good"] += 1
        else:
            users_dict[user]["bad"] += 1

print("Users total: ", len(users_dict))
bad_users = []
for key, value in users_dict.items():
    percentage_good = value["good"]/(value["good"] + value["bad"])
    if percentage_good < 0.5:
        bad_users.append(key)

print("Bad users:", len(bad_users))

Users total:  59
Bad users: 1


1 из 59 разметчиков на контрольных заданиях показали слишком плохое качество, чтобы учитывать их ответы для расчета метрики.

Отделяем контроль от основы, так как контрольные задания создавались отдельно и не должны учитываться при подсчете метрик. На контрольных заданиях есть `GOLDEN:result`. Также отсеиваем возможные баги Толоки, когда в строке может не быть задания - `INPUT:formula` содержит NaN.

In [6]:
assignments_no_control = assignments[assignments['GOLDEN:result'].isnull()]
assignments_no_control_no_null = assignments_no_control[assignments_no_control['INPUT:formula'].notnull()]

### Сбор ответов разметчиков и голосование

Собираем ответы голосования большинством для каждого задания.

In [7]:
from collections import defaultdict

text_dict = defaultdict(list)

for formula, user, out in zip(
    assignments_no_control_no_null["INPUT:formula"], assignments_no_control_no_null["ASSIGNMENT:worker_id"], 
    assignments_no_control_no_null["OUTPUT:result"]
    ):
    if user not in bad_users:
        text_dict[(formula)].append([
                user,
                {"out": out}
        ])

print(len(text_dict))

200


In [8]:
keys = list(text_dict.keys())
Counter([len(text_dict[keys[i]]) for i in range(len(keys))])

Counter({5: 198, 4: 2})

Только 2 задания с перекрытием 4 человека. Порог согласованности для них устанавливаем такой же, как и для заданий с перекрытием 5 человек - 3 человека для формирования большинства.

In [9]:
preds_full = {}
for i in range(len(keys)):
    ans = text_dict[keys[i]]
    lst = [ans[j][1]['out'] for j in range(len(ans))]
    cnt = Counter(lst)
    if len(lst) == 5:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most >= 3:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res
    elif len(lst) == 4:
        most = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][1]
        if most > 2:
            res = Counter([ans[j][1]['out'] for j in range(len(ans))]).most_common(1)[0][0]
            preds_full[keys[i]] = res

In [10]:
len(preds_full)

200

In [11]:
preds_full_df = pd.concat([pd.DataFrame(preds_full.keys(), columns=['formula']), pd.DataFrame(preds_full.values(), columns=['lb'])], axis=1)

Все задания согласованы.

### Сопоставление разметки и ground truth

Забираем задания из датасета с правильными ответами.

In [12]:
res_df = pd.read_csv('full_wa.tsv', sep='\t')

In [13]:
res_df = res_df.rename({'INPUT:formula': 'formula', 'GOLDEN:result': 'lb'}, axis=1)

После скачивания с Толоки в текстах рушится форматирование, потому нельзя просто сделать join двух табличек. Нужно убрать все "лишнее" форматирование сразу из двух табличек, чтобы остались только тексты, пунктуация и пробелы.

In [14]:
def format_text(text):
    text = (text.strip().replace('\n', ' ').replace('\t', ' ')
            .replace('\r', ' ').replace('  ', ' ').replace('  ', ' ')
            .replace('  ', ' '))
    return text

res_df['formula'] = res_df['formula'].apply(format_text)
preds_full_df['formula'] = preds_full_df['formula'].apply(format_text)

Делаем left join, чтобы соединить голосование и правильные метки для одних и тех же заданий.

In [15]:
new = res_df.merge(preds_full_df, on='formula', how='left')

In [17]:
new_valid = new[new['lb_y'].notna()].copy()
len(new_valid)

200

In [18]:
new_valid.head(1)

,formula,lb_x,lb_y
0,377 + 587,964,964


### Подсчет метрики

Так как в новом датасете есть ровно 200 строк, то мы ничего не потеряли. Мердж табличек прошел корректно и можно посчитать достоверные метрики.

In [19]:
new_valid['lb_y'] = new_valid['lb_y'].astype(int)

In [20]:
(new_valid['lb_x'] == new_valid['lb_y']).mean()

1.0

`Accuracy = 1.0`

### Анализ ошибок

Доля правильных ответов после голосования большинством составила ровно 1. Это значит, что дальнейший анализ ошибок результатов голосования не имеет смысла. Однако можно посмотреть на отдельные ответы разметчиков, чтобы понять были ли ошибки случайными или нет.

In [21]:
from scipy import stats

In [22]:
pool = pd.read_csv('assignments_from_pool_41864367__20-10-2023.tsv', sep='\t')

Убираем из пула контрольные задания.

In [23]:
pool = pool[pool['GOLDEN:result'].isna()]

In [24]:
pool1 = pool.rename({'INPUT:formula': 'formula', 'OUTPUT:result': 'lb'}, axis=1).iloc[:, [0, 1]].copy()

In [25]:
pool1.head(1)

,formula,lb
0,8920 + 2043,10963


Объединяем основной пул с правильными ответами.

In [26]:
pool1['formula'] = pool1['formula'].apply(lambda x: x.strip().replace('\n', ' ').replace('\t', ' ').replace('\r', ' ').replace('  ', ' ').replace('  ', ' ').replace('  ', ' '))

In [27]:
new = pool1.merge(res_df, on='formula', how='left')

Смотрим на количество ошибок.

In [28]:
(new['lb_x'] != new['lb_y']).sum()

24

Всего 24 ошибки... Задание слишком простое. С таким количеством ошибок никакой полезной информации извлечь не получится, так как их слишком мало, чтобы утверждать хоть что-то наверняка. Все же попробуем проверить, есть ли хоть какая-то зависимость ошибок от примеров.

In [29]:
new['cor'] = (new['lb_x'] == new['lb_y']).astype(int)

Имеет смысл оценить сложность примера по размеру слагаемых.

In [30]:
new['vars'] = new['formula'].apply(lambda x: len(x.split(' + ')[0]))

In [31]:
new.groupby('vars').agg(
    mean_correct = pd.NamedAgg(column='cor', aggfunc='mean'),
    median_correct = pd.NamedAgg(column='cor', aggfunc='median')
)

,mean_correct,median_correct
vars,,
1,1.000000,1.0
2,0.975862,1.0
3,0.978571,1.0
4,0.971875,1.0
5,0.975000,1.0


Нет зависимости между сложностью примера и ошибками. Рассмотрим только ошибки.

In [32]:
err = new[new['cor'] != 1].copy()

Возьмем разность правильного ответа и ответа разметчика.

In [33]:
err['diff'] = err['lb_y'] - err['lb_x']

In [34]:
err['diff'].value_counts()

diff
 1000     3
-100      2
 1        2
-10       1
-1000     1
 13500    1
 450      1
 100      1
-13500    1
 5        1
 12       1
-20       1
 45       1
 8        1
-7        1
 21       1
 81       1
 240      1
 3        1
 97100    1
Name: count, dtype: int64

Не похоже, что есть хоть какая-то зависимость между ошибками и условиями задач. Ошибок оцень мало, потому энтропии считать нет смысла. В самих ошибках нет никакой системы. Кажется, ошибки произошли целиком и полностью из-за невнимательности либо при ручном прорешивании примеров, либо при вбивании чисел в калькулятор. Отсюда ошибки ровно на 1, 10, 100, 1000 - разница в одну цифру с правильным ответом. Такие разницы (по модулю), как 97100, 13500, 81, 20 также связы с тем, что либо перепутана одна цифра в ответе, либо пропущена одна цифра в ответе, либо добавлена одна лишняя цифра в ответе. Причина всех этих ошибок - невнимательность, которая не связана с текстами заданий.